# Init

## Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/Smartech/MinaRumas')

import sys
sys.path.append('/content/drive/MyDrive/Smartech/MinaRumas')

!ls

## install

In [ ]:
!pip install -r requirements.txt

import ultralytics
ultralytics.checks()

## prueba

In [ ]:
# Process video, no --
if __name__ == "__main__":
    video_path = 'video/video_cam3_0422.mp4'
    output_path = generar_output_video(video_path)
    start_time_sec = 0# 10
    end_time_sec =2 # 120

    model_det_path = 'models/model_detection.pt'
    model_seg_path = 'models/model_segmentation.pt'

    camera_number = 3  # Cambiar según la cámara

    if camera_number ==1: ## donde la ruma estan esparcidas en un campo
      detection_zone = np.array([
          [20, 691], [18, 779], [414, 881], [709, 759],
          [1675, 1060], [1902, 890], [1902, 667], [704, 416]
          ], np.int32) # Cambiar las coordenadas de las zonas segun la camara, input de process_video (se puede setear en un .yml)
    elif camera_number ==2: # se puede ver la escalera o faja
      detection_zone = np.array([
          [1740,533],[1788,1075],[741,1074],[433,591],[400,480],[536,442],[1008,365],[1061,398],[1128,389],[1214,431],[1369,397]
          ], np.int32) # Cambiar las coordenadas de las zonas segun la camara, input de process_video (se puede setear en un .yml)
    elif camera_number ==3:
      detection_zone = np.array([
          [1862,767],[1443,633],[794,1077],[123,1072],[3,711],[4,517],[1122,256],[1893,450],[1860,534],[1873,611],[1855,689]
          ], np.int32) # Cambiar las coordenadas de las zonas segun la camara, input de process_video (se puede setear en un .yml)


    process_video(
        video_path, output_path, start_time_sec, end_time_sec,
        model_det_path, model_seg_path, detection_zone, camera_number
    )


## pruerba final

In [ ]:
import os
import cv2
import time
import json
import base64
import torch
import requests
import numpy as np
from datetime import datetime
from pathlib import Path
from collections import defaultdict, deque
from ultralytics import YOLO

# Librerías propias
from utils.paths import generar_output_video
from libs.main_functions import process_video, load_camera_config

In [ ]:


def main():
    start_time_sec = 0
    end_time_sec = 12
    camera_number = 3

    try:
        input_video, _, polygons, camera_sn, _ = load_camera_config(
            camera_number, config_path="mkdocs.yml"
        )
        output_video = generar_output_video(input_video)
        print(f"Procesando video: {input_video}")
        print(f"Salida: {output_video}")
    except ValueError as e:
        print(f"[Error] {e}")
        return

    model_det_path = 'models/model_detection.pt'
    model_seg_path = 'models/model_segmentation.pt'

    process_video(
        video_path=input_video,
        output_path=output_video,
        start_time_sec=start_time_sec,
        end_time_sec=end_time_sec,
        model_det_path=model_det_path,
        model_seg_path=model_seg_path,
        detection_zone=polygons,
        camera_number=camera_number,
        camera_serials=camera_sn
    )

    print("\n=== Procesamiento finalizado correctamente ===")

if __name__ == "__main__":
    main()


Numero de cámara: 3
Output video path: video/output/video_cam3_0422_processed.mp4
Output video path: video\output\video_cam3_0422\output_video_cam3_0422_20250712_1807.mp4
Video: 1920x1080 @ 30.00 FPS
Procesando frames 0 a 360
Procesados 0 frames
Rumas activas: 0
💾 Save alert local ejecutándose
✅ Alerta local guardada: movimiento_zona - 18:08:16
Nueva ruma creada: ID 1
Nueva ruma creada: ID 2
💾 Save alert local ejecutándose
📄 Imagen resumen de rumas guardada en: alerts_save\2025-07-12_local\18-08-45_ruma_summary_18.jpg
✅ Alerta local guardada: movimiento_zona - 18:08:45
💾 Save alert local ejecutándose
📄 Imagen resumen de rumas guardada en: alerts_save\2025-07-12_local\18-09-04_ruma_summary_28.jpg
✅ Alerta local guardada: movimiento_zona - 18:09:04
💾 Save alert local ejecutándose
📄 Imagen resumen de rumas guardada en: alerts_save\2025-07-12_local\18-09-12_ruma_summary_32.jpg
✅ Alerta local guardada: movimiento_zona - 18:09:12
Nueva ruma creada: ID 3
💾 Save alert local ejecutándose
📄 Imag

In [ ]:
"""
!grep "class " /content/drive/MyDrive/Smartech/MinaRumas/monitor/ruma_monitor.py
!python3 -m py_compile /content/drive/MyDrive/Smartech/MinaRumas/monitor/ruma_monitor.py
print("ok")
!python3 -m py_compile monitor/ruma_monitor.py
!python3 /content/drive/MyDrive/Smartech/MinaRumas/monitor/ruma_monitor.py
"""